In [1]:
import json
from shapely.geometry import shape, GeometryCollection, Point
from typing import Tuple
#Documentation: https://shapely.readthedocs.io/en/stable/

In [2]:
#read geojson file for wojewodztwo level
with open('../../data/data_auxiliary/wojewodztwa-max.geojson', "r", encoding="utf-8") as f:
     wojewodztwa = json.load(f)
#read geojson file for powiat level
with open('../../data/data_auxiliary/powiaty-max.geojson', "r", encoding="utf-8") as f:
     powiaty = json.load(f)

In [3]:
def look_up(longitude: float, latitude:float ) -> Tuple[str,str] :
    
    point: Point = Point(longitude, latitude)
    
    wojewodztwo: str = 'not in PL'
    powiat: str = 'not in PL'
    
    for feature in wojewodztwa['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            wojewodztwo = feature['properties']['nazwa']
    
    for feature in powiaty['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            powiat = feature['properties']['nazwa']
        
    return wojewodztwo, powiat


In [4]:
info: Tuple[str,str] = look_up(20.975166, 50.173166 )

In [5]:
print(info)

('małopolskie', 'powiat dąbrowski')


# Checking powiat for weather stations

In [6]:
import pandas as pd
stations = pd.read_csv("../../data/data_processed/processed_weather_data_in_CSV_files/stations_with_powiat_voivod.csv")

In [7]:
for ind, i in enumerate(zip(stations['LON'],stations['LAT'],stations['state'],stations['county'])):
    pow_voi = look_up(i[0],i[1])
    pow_voi_old = ( i[2].split(' ')[1] if len(i[2].split(' ')) == 2 else i[2], i[3])
    if pow_voi[1] != pow_voi_old[1]:
        stations.loc[ind,('county')] = pow_voi[1]
        print(f"LAT: {i[1]}, LON: {i[0]}, OLD VOIVOD/POWIAT: {pow_voi_old[0],pow_voi_old[1]}, NEW VOIVOD/POWIAT: {pow_voi[0],pow_voi[1]}")

LAT: 50.78333333, LON: 15.78333333, OLD VOIVOD/POWIAT: ('dolnośląskie', 'powiat karkonoski'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat jeleniogórski')
LAT: 51.96666667, LON: 20.16666667, OLD VOIVOD/POWIAT: ('łódzkie', 'Powiat skierniewicki'), NEW VOIVOD/POWIAT: ('łódzkie', 'powiat Skierniewice')
LAT: 52.28333333, LON: 20.96666667, OLD VOIVOD/POWIAT: ('mazowieckie', 'Warszawa'), NEW VOIVOD/POWIAT: ('mazowieckie', 'powiat Warszawa')
LAT: 52.21944444, LON: 20.99916667, OLD VOIVOD/POWIAT: ('mazowieckie', 'Warszawa'), NEW VOIVOD/POWIAT: ('mazowieckie', 'powiat Warszawa')
LAT: 54.38333333, LON: 18.46666667, OLD VOIVOD/POWIAT: ('pomorskie', 'Gdańsk County'), NEW VOIVOD/POWIAT: ('pomorskie', 'powiat Gdańsk')
LAT: 54.33333333, LON: 18.93333333, OLD VOIVOD/POWIAT: ('pomorskie', 'Gdańsk County'), NEW VOIVOD/POWIAT: ('pomorskie', 'powiat Gdańsk')
LAT: 50.90027778, LON: 15.78888889, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Gó

In [8]:
import random

for ind, i in enumerate(zip(stations['LON'],stations['LAT'],stations['state'],stations['county'])):
    pow_voi = (i[2], i[3])
    while pow_voi[1] == 'not in PL':
        new_coords = (i[0] + 0.1*(2*random.random()-1), i[1] + 0.1*(2*random.random()-1))
        pow_voi = look_up(*new_coords)
        if pow_voi != ('not in PL', 'not in PL'):
            print(i, "->", pow_voi)
            stations.loc[ind, ('state','county')] = ['województwo ' + pow_voi[0], pow_voi[1]]

(14.53777778, 52.36527778, 'Brandenburg', 'not in PL') -> ('lubuskie', 'powiat słubicki')
(15.73, 50.73, 'Severovýchod', 'not in PL') -> ('dolnośląskie', 'powiat jeleniogórski')


In [9]:
# add isUrban feature and get rid of the redundant Voivodeship
stations.drop(columns='Voivodeship',inplace=True)
stations['isUrban'] = [1 if x[7].isupper() else 0 for x in stations['county']]

In [10]:
stations.to_csv("../../data/data_processed/processed_weather_data_in_CSV_files/stations_with_powiat_voivod_GEOJSON.csv", index=False)

# Checking powiat for air quality stations

In [11]:
aq_stations = pd.read_csv("../../data/data_processed/merged_air_quality_data_CSV/thayes_air_quality_stations_with_powiat_missing3.csv")
aq_stations.head()

,No,Station code,International code,Station name,Old station code,Launching date,Closing date,Station type,Region type,Station model,Voivodeship,City,Address,WGS84 φ N,WGS84 λ E,county,STANAME
0,1,DsBialka,NaN,Białka,DsBialka,1990-01-03,2005-12-31,industrial,suburban,container,DOLNOŚLĄSKIE,Białka,NaN,51.197783,16.117390,powiat legnicki,BIALKA
1,2,DsBielGrot,NaN,Bielawa - ul. Grota Roweckiego,DsBielGrot,1994-01-02,2003-12-31,background,urban,in-house,DOLNOŚLĄSKIE,Bielawa,ul. Grota Roweckiego 6,50.682510,16.617348,powiat dzierżoniowski,BIELAWA
2,3,DsBogatFrancMOB,PL0602A,Bogatynia Mobil,DsBogatMob,2015-01-01,2015-12-31,background,urban,mobile,DOLNOŚLĄSKIE,Bogatynia,ul. Francuska/Kręta,50.940998,14.916790,powiat zgorzelecki,BOGATYNIA
3,4,DsBogChop,PL0315A,Bogatynia - Chopina,DsBogChop,1996-01-01,2013-12-31,industrial,urban,container,DOLNOŚLĄSKIE,Bogatynia,ul. Chopina 35,50.905856,14.967175,powiat zgorzelecki,BOGATYNIA
4,5,DsBogZatonieMob,PL0576A,Bogatynia - Mobil,DsBogZatonieMob,2012-01-01,2012-12-31,industrial,urban,mobile,DOLNOŚLĄSKIE,Bogatynia,"ul. Konrada, Zatonie",50.943245,14.913327,powiat zgorzelecki,BOGATYNIA


In [12]:
#see whether stations with missing coordinates can be safely disregarded
import datetime
startdate = datetime.datetime.strptime('Jan 01 00:00:00 2017', "%b %d %H:%M:%S %Y")
missing_coords = aq_stations.loc[aq_stations['WGS84 λ E'] == -999.0][['Closing date','WGS84 λ E','WGS84 φ N']]
missing_coords['Closing date'] = pd.to_datetime(missing_coords['Closing date'])
if all(missing_coords['Closing date'] < startdate) == True:
    print(f"{len(missing_coords['Closing date'])} stations with missing coordinates were closed before 2017")

76 stations with missing coordinates were closed before 2017


In [13]:
#drop rows with coordinates -999.0
aq_stations = aq_stations[aq_stations['WGS84 λ E'] != -999.0]

In [14]:
for i in zip(aq_stations['WGS84 λ E'],aq_stations['WGS84 φ N'],aq_stations['Voivodeship'],aq_stations['county'], aq_stations.index):
    pow_voi = look_up(i[0],i[1])
    pow_voi_old = ( i[2].lower(), i[3])
    if pow_voi[1] != pow_voi_old[1]:
        aq_stations.loc[i[4],('Voivodeship','county')] = pow_voi[0], pow_voi[1]
        print(f"LAT: {i[1]}, LON: {i[0]}, OLD VOIVOD/POWIAT: {pow_voi_old[0],pow_voi_old[1]}, NEW VOIVOD/POWIAT: {pow_voi[0],pow_voi[1]}")

LAT: 50.913433000000005, LON: 15.765608, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Góra')
LAT: 50.871214, LON: 15.700947, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Góra')
LAT: 50.862966, LON: 15.681788, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Góra')
LAT: 50.861853, LON: 15.678760999999998, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Góra')
LAT: 50.897497, LON: 15.736629, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Góra')
LAT: 50.897497, LON: 15.736629, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia Góra County'), NEW VOIVOD/POWIAT: ('dolnośląskie', 'powiat Jelenia Góra')
LAT: 50.865874, LON: 15.679301, OLD VOIVOD/POWIAT: ('dolnośląskie', 'Jelenia

In [15]:
#change remaining voivodeships to lowercase and save the results
#add isUrban feature
aq_stations['Voivodeship'] = aq_stations['Voivodeship'].apply(lambda x: x.lower())
aq_stations['isUrban'] = [1 if x[7].isupper() else 0 for x in aq_stations['county']]
aq_stations.to_csv('../../data/data_processed/merged_air_quality_data_CSV/air_quality_stations_with_powiat_GEOJSON.csv', index=False)